In [27]:
# Importing libraries
import cv2
import numpy as np

In [28]:
# Helper Functions
# Function to read movie data from a CSV file
def read_from_csv(csv_f):
    with open(csv_f, 'r') as f:
        movie_data = {}
        for line in f.readlines():
            line_split = line.strip().split(",")
            year = line_split[-1].split("|")
            keywords = line_split[-2].split("|")
            tagline = line_split[-3].split("|")
            director = line_split[-4].split("|")
            cast = line_split[-5].split("|")
            movie = line_split[0].upper()
            movie_data[movie] = [year, keywords, tagline, director, cast]
    return movie_data

# Function to get random movie information
def get_movie_info(movies_data):
    movies_list = list(movies_data.keys())
    movie = np.random.choice(movies_list, 1)[0].upper()
    movie_info = movies_data[movie]
    return movie, movie_info

# Function to select random hints for the movie
def select_hints(movie_info):
    # Randomly select 3 types of hints to display
    hints_index = list(np.random.choice(5, 3, replace=False))
    hints = []
    hints_labels = ["Release Year", "Keyword", "Tagline", "Director", "Cast"]
    labels = []
    for hint_index in hints_index:
        hint = np.random.choice(movie_info[hint_index], 1)[0].upper()
        hints.append(hint)
        labels.append(hints_labels[hint_index].upper())
    return hints, labels

# Function to load the game canvas image
def get_canvas(canvas_file):
    img = cv2.imread(canvas_file, 1)
    return img

# Function to draw the incorrect attempt text
def draw_wrong(img, incorrect_attempts):
    cv2.putText(img, f"WRONG {incorrect_attempts + 1}/6", (380, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    return img

# Function to draw hint on the canvas
def draw_hint(img, hints, labels, incorrect_attempts):
    x, y = 20, 30
    if incorrect_attempts == 0:
        return img
    elif incorrect_attempts <= 1:
        index = 0
    elif incorrect_attempts <= 3:
        index = 1
    elif incorrect_attempts <= 6:
        index = 2

    cv2.putText(img, f"HINT: {labels[index]}", (x, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 1)
    cv2.putText(img, hints[index], (x, y + 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 1)
    return img

# Function to draw the right guess text
def draw_right(img):
    cv2.putText(img, "RIGHT", (380, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    return img

# Function to draw the lost game text
def draw_lost(img):
    cv2.putText(img, "YOU LOST", (380, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    return img

# Function to draw the won game text
def draw_won(img):
    cv2.putText(img, "YOU WON", (380, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    return img

# Function to draw the invalid input text
def draw_invalid(img):
    cv2.putText(img, "INVALID INPUT", (300, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    return img

# Function to draw the already used character text
def draw_reuse(img):
    cv2.putText(img, "ALREADY USED", (300, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    return img

# Function to draw the characters that have been used
def draw_used_chars(img, chars_entered, letter):
    cv2.putText(img, "Letters used:", (300, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    y, x, count = 120, 350, 0
    for char in chars_entered:
        if count == 10:
            x += 50
            y = 120
        color = (0, 0, 255) if char == letter else (0, 0, 0)
        cv2.putText(img, char, (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        y += 20
        count += 1
    return img

# Function to get the coordinates of each character in the movie title
def get_char_coords(movie):
    x_coord, y_coord = 100, 400
    char_ws, char_hs = [], []

    for char in movie:
        char_width, char_height = cv2.getTextSize(char, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
        char_ws.append(char_width)
        char_hs.append(char_height)

    max_char_h, max_char_w = max(char_hs), max(char_ws)
    char_rects = []

    for width in char_ws:
        rect_coord = [(x_coord, y_coord - max_char_h), (x_coord + max_char_w, y_coord)]
        char_rects.append(rect_coord)
        x_coord += max_char_w

    return char_rects

# Function to draw blank rectangles for each character in the movie title
def draw_blank_rects(movie, char_rects, img):
    for i, char in enumerate(movie):
        top_left, bottom_right = char_rects[i]
        if not char.isalpha() or ord(char) < 65 or ord(char) > 122 or (ord(char) > 90 and ord(char) < 97):
            cv2.putText(img, char, (top_left[0], bottom_right[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.rectangle(img, top_left, bottom_right, (0, 0, 255), thickness=1, lineType=cv2.LINE_8)
    return img

# Function to check if all characters in the movie title have been guessed
def check_all_chars_found(movie, chars_entered):
    chars_to_be_checked = [char for char in movie if char.isalpha()]
    for char in chars_to_be_checked:
        if char not in chars_entered:
            return False
    return True

# Functions to draw different parts of the hangman figure
def draw_circle(img):
    cv2.circle(img, (190, 160), 40, (0, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    return img

def draw_back(img):
    cv2.line(img, (190, 200), (190, 320), (0, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    return img

def draw_left_hand(img):
    cv2.line(img, (190, 240), (130, 200), (0, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    return img

def draw_right_hand(img):
    cv2.line(img, (190, 240), (250, 200), (0, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    return img

def draw_left_leg(img):
    cv2.line(img, (190, 320), (130, 360), (0, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    return img

def draw_right_leg(img):
    cv2.line(img, (190, 320), (250, 360), (0, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    return img

# Function to draw the hangman figure based on the number of tries
def draw_hangman(img, num_tries):
    if num_tries == 1:
        return draw_circle(img)
    elif num_tries == 2:
        return draw_back(img)
    elif num_tries == 3:
        return draw_left_hand(img)
    elif num_tries == 4:
        return draw_right_hand(img)
    elif num_tries == 5:
        return draw_left_leg(img)
    elif num_tries == 6:
        return draw_right_leg(img)
    else:
        return img

# Function to reveal the entire movie title
def reveal_movie(movie, img, char_rects):
    for i in range(len(movie)):
        top_left, bottom_right = char_rects[i]
        cv2.putText(img, movie[i], (top_left[0], bottom_right[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    return img

# Function to display the guessed letter in the movie title
def display_letter(img, letter, movie, char_rects):
    for i in range(len(movie)):
        if movie[i] == letter:
            top_left, bottom_right = char_rects[i]
            cv2.putText(img, movie[i], (top_left[0], bottom_right[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    return img

In [29]:
# File paths for the movie data CSV and the blank canvas image
movie_csv = "movies-list-short.csv"
canvas = "blank-canvas.png"

# Read movie data from the CSV file
movies_data = read_from_csv(movie_csv)

# Get a random movie and its associated information
movie, movie_info = get_movie_info(movies_data)
print(movie)

DONNIE DARKO


In [30]:
# Select random hints for the movie
hints,labels = select_hints(movie_info)

# Load the game canvas image and get coordinates for each character in movie title
img = get_canvas(canvas)
char_rects = get_char_coords(movie)

# Draw the blank rectangles for the movie title
img = draw_blank_rects(movie,char_rects,img)

In [31]:
# Display the canvas image in the full-screen window
cv2.namedWindow("Hangman", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Hangman",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)

cv2.imshow("Hangman",img)

In [32]:
chars_entered = []
incorrect_attempts = 0

img_copy = img.copy()

In [33]:
while True:
    # Make a copy of the original canvas image
    img = img_copy.copy()
    
    # Draw hints on the canvas
    img = draw_hint(img, hints, labels, incorrect_attempts)
    
    # Check if the player has reached the maximum number of incorrect attempts
    if incorrect_attempts >= 6:
        img = draw_lost(img)
        break
    
    # Check if all characters in the movie title have been guessed correctly
    elif check_all_chars_found(movie, chars_entered):
        img = draw_won(img)
        break
    
    else:
        # Wait for the player to input a letter
        letter = cv2.waitKey(0) & 0xFF
        
        # Check if the input is valid (a letter from A-Z or a-z)
        if letter < 65 or letter > 122 or (letter > 90 and letter < 97):
            img = draw_invalid(img)
            cv2.imshow("Hangman", img)
            continue
        else:
            # Convert the input letter to uppercase
            letter = chr(letter).upper()
        
        # Check if the letter has already been guessed
        if letter in chars_entered:
            img = draw_reuse(img)
            img = draw_used_chars(img, chars_entered, letter)
            cv2.imshow("Hangman", img)
            continue
        else:
            chars_entered.append(letter)
            
            # Check if the guessed letter is in the movie title
            if letter in movie:
                img = draw_right(img)
                img = displayLetter(img, letter, movie, char_rects)
                img_copy = displayLetter(img_copy, letter, movie, char_rects)
            else:
                img = draw_wrong(img, incorrect_attempts)
                incorrect_attempts += 1
    
    # Draw the used characters and hangman figure on the canvas
    img = draw_used_chars(img, chars_entered, letter)
    img = draw_hangman(img, incorrect_attempts)
    img_copy = draw_used_chars(img_copy, chars_entered, letter)
    img_copy = draw_hangman(img_copy, incorrect_attempts)
    
    # Display the updated canvas image
    cv2.imshow("Hangman", img)
    # cv2.waitKey(0)

In [34]:
img = revealMovie(movie,img,char_rects)
cv2.imshow("Hangman",img)
cv2.waitKey(0)

cv2.destroyAllWindows()